## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

Импортируем библиотеку pandas и открываем таблицу с данными, выводим на экран первые 5 строк таблицы:

In [1]:
import pandas as pd
solvency = pd.read_csv('/datasets/data.csv') #solvency - платежеспособность (англ.)
solvency.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Запрашиваем общую информацию о массиве данных. 
Сразу обращаем внимание на проблемы с неоднородностью данных в столбце education (буквы в разном регистре):

In [2]:
solvency.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Получаем всего 21525 записей, пропуски имеются в двух столбцах: days_employed и total_income.
В данных столбцах равное количество пропусков, в дальнейшем необходимо рассмотреть взаимосвязь этих параметров.

Выведем на экран уникальные значения категориальных параметров, которые должны иметь ограниченный набор значений:

In [3]:
print('Образование:', solvency['education'].unique())
print()
print('Семейное положение: ', solvency['family_status'].unique())
print()
print('Пол: ', solvency['gender'].unique())
print()
print('Занятость: ', solvency['income_type'].unique())

Образование: ['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']

Семейное положение:  ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']

Пол:  ['F' 'M' 'XNA']

Занятость:  ['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']


### Вывод

1. В качестве вводных данных имеем таблицу, состоящую из 12 стобцов и 21525 строк. Данные в таблице относятся к одному из трех типов: integer, float или object. Столбцы отображают 5 категориальных и 7 количественных показателей.
2. В данных имеются пропуски в двух столбцах, в которых содержится информация о трудовом стаже и доходе за месяц (количественные показатели), причем количество пропусков для них равно между собой. 
3. В столбце с информацией об образовании наблюдается неоднородность в данных, заключающаяся в наличии символов в разном регистре. Причем в результате запроса уникальных значений для данного столбца стало ясно, что проблема еще масштабнее, чем могло показаться на первывй взгляд, что является причиной большого количества дубликатов.
4. В столбце с информацией об общем трудовом стаже набюлюдаются отрицательные значения, что физически невозможно (причина может быть в том, что при расчете трудового стажа методом вычитания текущей даты и даты приема на работу, блыли перепутаны местами уменьшаемое и вычитаемое).
5. В результате запроса уникальных значений некоторых параметров обнаружились неочевидные пропуски в данных: значение 'XNA' в графе 'пол' (пол не определен).

### Шаг 2. Предобработка данных

### Обработка пропусков

Посчитаем количество пропусков в таблице:

In [4]:
solvency.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Наблюдаем пропуски в стобцах "общий стаж работы" и "суммарный доход". Выведем на экран и изучим респондентов, имеющих пропуски по этим столбцам:

In [5]:
solvency_na = solvency[solvency['days_employed'].isna()==True]
solvency_na.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Можно бы было предположить, что информация о стаже и доходе будет отсутствовать у тех, у кого их еще/в данный момент нет (студенты). Однако просмотр информации по строкам с пропусками в исследуемых параметрах явно говорит о разной возрастной принадлежности респондентов, попавших в список, а также о их различном образовании и типе занятости.

Выясним, как распределились строки с отсуствующими значениями в зависимости от типа занятости:

In [6]:
solvency_na['income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

In [7]:
solvency_na['income_type'].value_counts()['сотрудник']

1105

Вопреки изначальному предположению, среди респондентов с пропущенными данными нет ни одного студента, при этом есть представители групп "сотрудники", "компаньоны", "пенсионеры" и "предприниматели".

Выясним, какова доля различных типов занятости среди общего количества респондентов с пропусками в данных:

In [9]:
for inc_type in solvency_na['income_type'].unique():
    print('Доля группы "{}" среди всех, не указавших свои данные: {:.2%}'.format(
        inc_type,
        solvency_na['income_type'].value_counts()[inc_type] / solvency_na.shape[0]))

Доля группы "пенсионер" среди всех, не указавших свои данные: 19.00%
Доля группы "госслужащий" среди всех, не указавших свои данные: 6.76%
Доля группы "компаньон" среди всех, не указавших свои данные: 23.37%
Доля группы "сотрудник" среди всех, не указавших свои данные: 50.83%
Доля группы "предприниматель" среди всех, не указавших свои данные: 0.05%


Определим долю представителей вышеуказанных типов занятости среди общего количества записей:

In [10]:
solvency['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [11]:
print('Доля сотрудников среди всех респондентов: {:.2%}'.format(11119 / 21525))
print('Доля компаньонов среди всех респондентов: {:.2%}'.format(5085 / 21525))
print('Доля пенсионеров среди всехреспондентов: {:.2%}'.format(3856 / 21525))
print('Доля госслужащих среди всех респонлентов: {:.2%}'.format(1459 / 21525))
print('Доля предпринимателей среди всех респондентов: {:.4%}'.format(2 / 21525))

Доля сотрудников среди всех респондентов: 51.66%
Доля компаньонов среди всех респондентов: 23.62%
Доля пенсионеров среди всехреспондентов: 17.91%
Доля госслужащих среди всех респонлентов: 6.78%
Доля предпринимателей среди всех респондентов: 0.0093%


Таким образом мы выяснили, что доля респондентов по типам  занятости, не заполнивших данные, соответствует доле респондентов, относящихся к данному типу занятости среди всего объема данных с отклонением около 1%. 
Можно сделать вывод, что пропуски в данных имели совершенно случайный характер.
Исключение составляет только тип занятости "предприниматель" ввиду малого количества представителей этого типа.

Заполним пропущенные значения средними показателями для каждого типа занятости.
Для этого сперва избавимся от отрицательных значений:

In [12]:
solvency['days_employed'] = solvency['days_employed'].apply(abs) 
#применяем ко всем значениям столбца days_employed 
#функцию abs, возвращающую модуль значения

Рассчитаем средние значения стажа работы и ежемесячного дохода для каждой группы, в которой были встречены пропуски:

In [13]:
#Для сотрудника:
employee_mean_income = solvency[solvency['income_type']=='сотрудник']['total_income'].mean()
employee_mean_day = solvency[solvency['income_type']=='сотрудник']['days_employed'].mean()

#Для компаньона:
companion_mean_income = solvency[solvency['income_type']=='компаньон']['total_income'].mean()
companion_mean_day = solvency[solvency['income_type']=='компаньон']['days_employed'].mean()

#Для пенсионера:
retiree_mean_income = solvency[solvency['income_type']=='пенсионер']['total_income'].mean()
retiree_mean_day = solvency[solvency['income_type']=='пенсионер']['days_employed'].mean()

#Для госслужащего:
state_employee_mean_income = solvency[solvency['income_type']=='госслужащий']['total_income'].mean()
state_employee_mean_day = solvency[solvency['income_type']=='госслужащий']['days_employed'].mean()

Количество исследований, описывающих данные для типа занятости "предприниматель" недостаточно для формулировки предположения о возможном доходе и стаже:

In [14]:
solvency[solvency['income_type'] == 'предприниматель']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
5936,0,NaN,58,высшее,0,женат / замужем,0,M,предприниматель,0,NaN,покупка жилой недвижимости
18697,0,520.848083,27,высшее,0,гражданский брак,1,F,предприниматель,0,499163.144947,на проведение свадьбы


Заполним пропуски полученными средними значениями:

In [15]:
solvency.loc[solvency['income_type'] == 'сотрудник', 'days_employed'] = solvency.loc[solvency['income_type'] == 'сотрудник', 'days_employed'].fillna(employee_mean_day)
solvency.loc[solvency['income_type'] == 'сотрудник', 'total_income'] = solvency.loc[solvency['income_type'] == 'сотрудник', 'total_income'].fillna(employee_mean_income)

solvency.loc[solvency['income_type'] == 'компаньон', 'days_employed'] = solvency.loc[solvency['income_type'] == 'компаньон', 'days_employed'].fillna(companion_mean_day)
solvency.loc[solvency['income_type'] == 'компаньон', 'total_income'] = solvency.loc[solvency['income_type'] == 'компаньон', 'total_income'].fillna(companion_mean_income)

solvency.loc[solvency['income_type'] == 'пенсионер', 'days_employed'] = solvency.loc[solvency['income_type'] == 'пенсионер', 'days_employed'].fillna(retiree_mean_day)
solvency.loc[solvency['income_type'] == 'пенсионер', 'total_income'] = solvency.loc[solvency['income_type'] == 'пенсионер', 'total_income'].fillna(retiree_mean_income)

solvency.loc[solvency['income_type'] == 'госслужащий', 'days_employed'] = solvency.loc[solvency['income_type'] == 'госслужащий', 'days_employed'].fillna(state_employee_mean_day)
solvency.loc[solvency['income_type'] == 'госслужащий', 'total_income'] = solvency.loc[solvency['income_type'] == 'госслужащий', 'total_income'].fillna(state_employee_mean_income)



Итак, у нас осталось по одному пропуску в столбцах "стаж" и "ежемесячный доход". Эти характеристики принадлежат респонденту с типом занятости "предприниматель". 
У нас нет достаточной информации для того, чтобы заполнить пропуски: для госслужащих и пенсионеров доход находится в неким рамках, которые можно определить. По сотрудникам самая большая выборка, что при большей возможной дифференциации дохода позволяет найти средние значения.
Предприниматель - более обширная категория работников, в которой могли бы оказаться объекты как с самым высоким, тамк и самым низким доходом среди всех данных.
Поскольку основной характеристикой, влияющей на платежеспособность, является именно доход, а запись с неопределенностью у нас всего одна, целесообразным представляется исключить эту запись из данных, это почти не отразится на общем объеме.

In [16]:
solvency = solvency.dropna()

In [17]:
#проверим количество пропусков в таблице после проведенной работы
solvency.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Вывод

Итак, мы избавились от пропусков в таблице. Изначальное предположение состояло в том, что пропуски - отсутствующие значения, а не пропущенные, в таком случае пропуски можно было бы заменить на "0", однако данная гипотеза не подтвердилась.
Мы провели анализ и выяснили, что пропуски распределились равномерно и случайным образом среди 5 групп объектов по признаку "тип занятости".
Поскольку основным фактором, влияющим на платежеспособность респондентов, является их доход, а границы дохода находятся в непосредственной взаимосвязи с типом занятости (укрупненной группой характеристики "должность"), мы заполнили пропуски по каждой группе типа занятости средними значениями для этих групп.
Исключение составил тип занятости "предприниматель", который включал в себя всего две записи, поэтому запись с пропущенными данными для категории "предприниматели" мы решили удалить.

Также для корректного расчета среднего значения стажа мы по ходу дела избавились от отрицательных показателей в графе "стаж", предположив, что причиной появления отрицательных значений стала ошибка в формуле, и по модулю значения верны.

### Замена типа данных

Заменим вещественный тип данных в нашей таблице на целочисленный:

In [18]:
solvency['days_employed'] = solvency['days_employed'].astype('int')
solvency['total_income'] = solvency['total_income'].astype('int')

In [19]:
solvency.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 12 columns):
children            21524 non-null int64
days_employed       21524 non-null int64
dob_years           21524 non-null int64
education           21524 non-null object
education_id        21524 non-null int64
family_status       21524 non-null object
family_status_id    21524 non-null int64
gender              21524 non-null object
income_type         21524 non-null object
debt                21524 non-null int64
total_income        21524 non-null int64
purpose             21524 non-null object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


Для изменения типа данных был использован метод "astype", поскольку нам нужно перевести значения в тип int из типа float, а стандартный метод to_numeric преобразует тип либо во float, либо в integer в зависимости от исходных значений.

Распечатаем первые 5 строк нашей таблицы:

In [20]:
solvency.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


Как мы видим, в таблице остались только целочисленные значения в столбцах с количественными показателями.
Проверим, какие типы данных остались в нашем датафрейме:

In [21]:
solvency.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 12 columns):
children            21524 non-null int64
days_employed       21524 non-null int64
dob_years           21524 non-null int64
education           21524 non-null object
education_id        21524 non-null int64
family_status       21524 non-null object
family_status_id    21524 non-null int64
gender              21524 non-null object
income_type         21524 non-null object
debt                21524 non-null int64
total_income        21524 non-null int64
purpose             21524 non-null object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


### Вывод

После замены типа данных в нашей таблице остались только количественные целочисленные значения и категориальные значения формата object.

### Обработка дубликатов

Сперва избавимся от дубликатов в столбце "образование", вызванных наличием символов в разном регистре:

In [22]:
#определяем фунцию low_letters, в качестве аргумента принимающую значение ячейки из столбца
def low_letters(column):
#фунция возвращает исходное значение после применения к нему методов .str и .lower()
    return column.str.lower()

    

solvency[['education']] = solvency[['education']].apply(low_letters)

Убедимся, что проблема с регистровыми дубликатами решена:

In [23]:
solvency['education'].value_counts()

среднее                15233
высшее                  5259
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Выясним, сколько дубликатов содержится в таблице после проделанной предобработки данных:

In [24]:
solvency.duplicated().sum()

71

Удалим дубликаты и убедимся, что в таблице остались только уникальные строки:

In [25]:
solvency = solvency.drop_duplicates()
solvency.duplicated().sum()

0

Посмотрим, сколько уникальных наблюдений осталось в нашей таблице после удаления дубликатов:

In [26]:
solvency.shape

(21453, 12)

### Вывод

В результате удаления дубликатов мы избавились от 71 повторяющейся строки. После удаления одного наблюдения в разделе с обработкой пропусков и после удаления дубликатов, выполненного в этом разделе, из 21525 изначальных наблюдений у нас осталась 21453.

### Лемматизация

Для проведения анализа целей взятия кредита выполним лемматизацию значений столбца "purpose" с помощью метода Mystem библиотеки pymystem3, он является предпочтительнее стемминга,поскольку стемминг находит общие части в слове, а не основу слова:

In [27]:
from pymystem3 import Mystem
from collections import Counter
m = Mystem()

purposes = []
for purpose in solvency['purpose']:
    lemmas = m.lemmatize(purpose)
    purposes += lemmas
print(Counter(purposes))

Counter({' ': 33568, '\n': 21453, 'недвижимость': 6350, 'покупка': 5896, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2324, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1229, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'подержанный': 486, 'подержать': 478, 'приобретение': 461, 'профильный': 436})


In [28]:
#создадим таблицу, в которой отобразим основные цели взятия кредита
purpose_df = pd.DataFrame({'purpose' : ['недвижимость', 'жилье', 'автомобиль', 
                                        'образование', 'свадьба', 'строительство',
                                        'ремонт'], 'quantity' : 
                           [6350, 4460, 4306, 4013, 2324, 1878, 607]})

purpose_df

,purpose,quantity
0,недвижимость,6350
1,жилье,4460
2,автомобиль,4306
3,образование,4013
4,свадьба,2324
5,строительство,1878
6,ремонт,607


In [29]:
#напишем и применим к датафрейму функцию, заменяющую слова в формулировке цели на их леммы
def purpose_lemmas(row):
    purpose_lemmas = m.lemmatize(row)
    purpose_lemmas = ''.join(purpose_lemmas)
    return purpose_lemmas.rstrip('\n')
    
    
solvency['purpose'] = solvency['purpose'].apply(purpose_lemmas)

### Вывод

В результате лемматизации мы получили список лемм-категорий с их весовыми значениями по целям взятия кредита.
Выделили 7 основных категорий: недвижимость, жилье, покупка автомобиля, образование, организация свадьбы, строительство и проведение ремонта.
Следует обратить внимание на группу недвижимость - жилье - строительство, поскольку недвижимость может быть как коммерческой, так и жилой, а строительство по конечному результату вложения денег эквивалентно покупке жилой недвижимости.

### Категоризация данных

Для того, чтобы выполнить категоризацию по целям оформления кредита, выведем на экран полный список целей. Это поможет исключить дублирование целей, содержащих разные леммы:

Для того, чтобы укрупнить группы целей взятия кредита, объединим 7 полученных выше целей в группы: недвижимость, жилье, строительство - объединим в группу "недвижимость", это наиболее затратная группа в плане финансов. А саму группу "недвижимость" разобъем на две подргуппы - жилая и коммерческая. Образование, ремонт и свадьбу оставим без изменений.


In [30]:
solvency['purpose'].value_counts()

автомобиль                                972
свадьба                                   791
на проведение свадьба                     768
сыграть свадьба                           765
операция с недвижимость                   675
покупка коммерческий недвижимость         661
операция с жилье                          652
покупка жилье для сдача                   651
операция с коммерческий недвижимость      650
жилье                                     646
покупка жилье                             646
покупка жилье для семья                   638
строительство собственный недвижимость    635
недвижимость                              633
операция со свой недвижимость             627
строительство жилой недвижимость          624
покупка недвижимость                      621
покупка свой жилье                        620
строительство недвижимость                619
ремонт жилье                              607
покупка жилой недвижимость                605
на покупка свой автомобиль        

Мы видим, что цель "ремонт жилья" содержит леммы "ремонт" и "жилье", при этом встречается 607 раз. "Жилье" встречается 4460 раз, а "ремонт" - 607 раз. Таким образом, лемма "ремонт" полностью включена во множество "жилье", и эту категорию можно удалить.
Далее: лемма "недвижимость" самая распространенная, и встречается 6350 раз. При этом недвижимость делится на жилую и коммерческую. При этом лема "коммерческий" встречается только в связке с леммой "недвижимость", таким образом, мы можем выделить из общего объема недвижимости коммерческую, а остаток объединить с леммой "жилье".
Лемма "строительство" также встречается только в связке с недвижимочстью, поэтому ее, как и "ремонт" можно исключить.

Таким образом, все цели из нашего массива данных можно разделить на 5 категорий: жилая недвижимость, коммерческая недвижимость, свадьба, автомобиль, образование.

На основании сделанных выше заключений добавим в нашу изначальную таблицу столбец, в который поместим укрупненную категорию цели взятия кредита.

In [31]:
solvency['purpose'].unique()

array(['покупка жилье', 'приобретение автомобиль',
       'дополнительный образование', 'сыграть свадьба',
       'операция с жилье', 'образование', 'на проведение свадьба',
       'покупка жилье для семья', 'покупка недвижимость',
       'покупка коммерческий недвижимость', 'покупка жилой недвижимость',
       'строительство собственный недвижимость', 'недвижимость',
       'строительство недвижимость', 'на покупка подержать автомобиль',
       'на покупка свой автомобиль',
       'операция с коммерческий недвижимость',
       'строительство жилой недвижимость', 'жилье',
       'операция со свой недвижимость', 'автомобиль',
       'заниматься образование', 'сделка с подержанный автомобиль',
       'получение образование', 'свадьба',
       'получение дополнительный образование', 'покупка свой жилье',
       'операция с недвижимость', 'получение высокий образование',
       'свой автомобиль', 'сделка с автомобиль', 'профильный образование',
       'высокий образование', 'покупка жилье 

Напишем функцию, которая заменит изначальное значение целей респондентов на укрупненные общие категории:

In [32]:
def purpose_group(row):
    #row = row['purpose']
    if 'автомобиль' in row:
        return 'автомобиль'
    if 'свадьба' in row:
        return 'свадьба'
    if 'образование' in row:
        return 'образование'
    if 'недвижимость' or 'жилье 'in row:
        if 'коммерческий' in row:
            return 'коммерческая недвижимость'
        return 'жилая недвижимость'
    else:
        return 'категория не определена'
    
solvency['purpose'] = solvency['purpose'].apply(purpose_group)
solvency['purpose'].value_counts()

жилая недвижимость           9499
автомобиль                   4306
образование                  4013
свадьба                      2324
коммерческая недвижимость    1311
Name: purpose, dtype: int64

### Вывод

В результате лемматизации целей взятия кредита и последующей категоризации полученных лемм, мы объединили все цели в 5 общих категорий: жилая недвижимость, автомобиль, образование, свадьба, коммерческая недвижимость.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Сперва выведем на экран уникальные значения нашего датасета по параметру "количество детей:

In [33]:
solvency['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

Как мы видим, в данных имеется 8 уникальных значений, причем одно значение отрицательное. Заменим отрицательное значение на положительное, равное по модулю:

In [34]:
solvency['children'] = solvency['children'].apply(abs) 

Проверим, не появилось ли в результате этого изменения новых дубликатов в данных:

In [35]:
solvency.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Дубликаты отсутствуют, продолжим обработку данных. Создадим датасет, в котором будут отражены в строках - количество детей, и в столбцах - количество респондентов с указанным количеством детей, а также количество респондентов, имеющих просрочки по кредитным платежам.

После этого добавим столбец "index", в котором рассчитаем долю респондентов по каждой категории, не имеющих просрочки по платежам.

Определим функцию children_function() и поместим код в тело функции.

In [36]:
def children_function():
    summ = solvency.groupby('children')['debt'].sum()
    count = solvency.groupby('children')['debt'].count()
    children_data = pd.DataFrame(data=dict(summ=summ, count=count)).reset_index()
    children_data['index'] = 1 - children_data['summ'] / children_data['count']
    return children_data
    
    
    
children_function()

,children,summ,count,index
0,0,1063,14090,0.924556
1,1,445,4855,0.908342
2,2,194,2052,0.905458
3,3,27,330,0.918182
4,4,4,41,0.902439
5,5,0,9,1.000000
6,20,8,76,0.894737


In [37]:
#индекс, показывающий долю респондентов, выплативших кредитные платежи в срок по всей выборке
general_debt_index = 1 - solvency['debt'].sum() / solvency['debt'].count()
print(general_debt_index)

0.9188458490653988


Выше мы сгруппировали всех респондентов по количеству детей и рассчитали для каждой группы общее количество записей в ней (count) , а также количество зафиксированных просрочек кредитных платежей (summ). После чего вычислили для каждой группы индекс, характеризующий вероятность того, что платеж НЕ будет просрочен.

Для сравнения также рассчитали этот индекс для всей совокупности в целом.

Также вызывает вопросы последняя строка в таблице children - это респонденты, у которых 20 детей. Учитывая отсутствие строк, соответствующих количеству детей, равному 6, 7, и так далее, а также видное невооруженным глазом уменьшение количества респондентов при увеличении количества детей от 0 до 5, и расположение по этому параметру 20-детных между 3-детными и 4-детными, мы можем с большой долей вероятности предположить, что и здесь имеем дело с ошибкой в данных, вызванной некорректным вводом, и истинное значение для этих наблюдений по количеству детей равняется 2. Произведем соответствующие изменения:

In [38]:
solvency['children'] = solvency['children'].replace(to_replace = 20, value = 2)
children_function()

,children,summ,count,index
0,0,1063,14090,0.924556
1,1,445,4855,0.908342
2,2,202,2128,0.905075
3,3,27,330,0.918182
4,4,4,41,0.902439
5,5,0,9,1.000000


В результате мы получили, что при общем индексе внесения в срок кредитного платежа, равном 0.919, индексы групп варьируются в интервале от 0.902 до 0.925 (для респондентов, имеющих 5 детей, индекс равен 1, однако это скорее является результатом недостаточной репрезентативности выборки: всего 9 наблюдений; при вероятности просрочки оплаты по кредиту менее, чем 1 к 10, отсутствие просрочек у 9 респондлентов из 9 не выходит за рамки общего правила).

Между наличием (количеством) детей и возвратом отстуствует какая-либо зависимость. Вероятность внесения кредитных платежей в срок варьируется в диапазоне от 90,2% до 92,5%.

### Вывод

Мы выяснили, что доля респондентов, не имеющих просрочек по кредитным платежам, одинакова для всех групп, объединенных по параметру "количество детей". Можно заключить, что количество детей никак не влияет на своевременность выплат по кредитам.

Однако следует иметь в виду, что факт наличия детей сам по себе не является достаточным основанием для оценки платежеспособности. Несовершеннолетние дети требуют от своих родителей дополнительных расходов, тогда как взрослые дети, имеющие стабильный заработок, наоборот могут финансово помогать родителям. Для более точной оценки важно знать также принадлежность детей к группе "несовершеннолетние" или "взрослые".

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [39]:
#выведем на экран все возможные уникальные значения параметра "семейный статус"
solvency['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

Мы имеем 5 различных вариантов, однако за разными названиями скрывается одинаковая суть: нам важно, одинок человек, или нет, и не интересны подробности его личной жизни. Таким образом, можно выделить две укрупненные группы значений: "одинокий" и "не одинокий" (есть семья).

In [40]:
def family_status_group(row):
    #row = row['purpose']
    if 'женат / замужем' in row:
        return 'есть семья'
    if 'гражданский брак' in row:
        return 'есть семья'
    if 'вдовец / вдова' in row:
        return 'одинокий / одинокая'
    if 'в разводе' in row:
         return 'одинокий / одинокая'
    if 'Не женат / не замужем' in row:
         return 'одинокий / одинокая'    
    else:
        return 'ошибка в определении семейного статуса'
    
solvency['family_status'] = solvency['family_status'].apply(family_status_group)
solvency['family_status'].value_counts()

есть семья             16489
одинокий / одинокая     4964
Name: family_status, dtype: int64

In [41]:
#соберем данные по семейному статусу в сводную таблицу:
family_pivot = solvency.pivot_table(values='debt', index='family_status', aggfunc=(sum, 'count')).reset_index()
family_pivot['index'] = 1 - family_pivot['sum'] / family_pivot['count']
family_pivot['delta%'] = (family_pivot['index'] - general_debt_index)*100
family_pivot

,family_status,count,sum,index,delta%
0,есть семья,16489,1319,0.920007,0.116143
1,одинокий / одинокая,4964,422,0.914988,-0.385794


### Вывод

Мы видим, что индекс выплат по кредиту в срок для респондентов с семьями и одиноких респондентов составляет 0.92 и 0.915 соответственно, а отклонение от среднего индекса составляет 0.11% и 0.39% соответственно.
Таким образом, и в данном случае можно говорить об отсутствии взаимосвязи между семейным положением и вероятностью того, что платеж по кредиту будет просрочен.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Наше изначальное предположение состояло в том, что доход является фактором, сильнее всего влияющим на вероятность просрочки платежа по кредиту. Исходя из предыдущих выводов может оказаться, что доход будет единственным значимым фактором. Проверим наше предположение:

In [42]:
ma = solvency['total_income'].max()
mi = solvency['total_income'].min()
me = int(solvency['total_income'].mean())
med = int(solvency['total_income'].median())

print('Максимальный доход: ', ma)
print('Минимальный доход: ', mi)
print('Средний доход: ', me)
print('Медианный доход: ', med)

Максимальный доход:  2265604
Минимальный доход:  20667
Средний доход:  167416
Медианный доход:  151876


In [43]:
#в случае необходимости с помощью кода ниже можно посмотреть на записи, соответствующие
#самым высоким и самым низким доходам

#solvency.sort_values(by='total_income', ascending=False).head()
#solvency.sort_values(by='total_income', ascending=False).tail()

Ежемесячный доход в нашей выборке варьируется в диапазоне от 20 667 до 2 265 604. Разделим наших респондентов на 5 категорий по доходу: от 20 до 50 тыс., от 50 до 100, от 100 до 300, от 300 до 1 млн., и от 1 млн. до 2 млн. 500 тыс.

In [44]:
def income_group(row):
    #row = row['purpose']
    if row >=20000 and row < 50000:
        return '20 000 - 50 000'
    if row >=50000 and row < 100000:
        return '50 000 - 100 000'
    if row >=100000 and row < 300000:
        return '100 000 - 300 000'  
    if row >=300000 and row < 1000000:
        return '300 000 - 1 000 000'
    if row >=1000000 and row < 2500000:
        return '1 000 000 - 2 500 000'
    else:
        return 'категория дохода не определена'
    
solvency['income_group'] = solvency['total_income']
solvency['income_group'] = solvency['income_group'].apply(income_group)
solvency['income_group'].value_counts()

100 000 - 300 000        15508
50 000 - 100 000          4091
300 000 - 1 000 000       1457
20 000 - 50 000            372
1 000 000 - 2 500 000       25
Name: income_group, dtype: int64

In [45]:
#создадим сводную таблицу по группам дохода, отобразим в ней общее количество респондентов, 
#количество просрочек по платежам, индекс вероятности внесения платежа в срок и
#отклонения от среднего индекса (в процентах)
income_pivot = solvency.pivot_table(values='debt', index='income_group', aggfunc=(sum, 'count')).reset_index()
income_pivot['index'] = 1 - income_pivot['sum'] / income_pivot['count']
income_pivot['delta%'] = (income_pivot['index'] - general_debt_index)*100
income_pivot

,income_group,count,sum,index,delta%
0,1 000 000 - 2 500 000,25,2,0.920000,0.115415
1,100 000 - 300 000,15508,1281,0.917397,-0.144838
2,20 000 - 50 000,372,23,0.938172,1.932619
3,300 000 - 1 000 000,1457,104,0.928620,0.977460
4,50 000 - 100 000,4091,331,0.919091,0.024484


### Вывод

По выбранным 5 категориям дохода индекс вероятности внесения платежа по кредиту в срок варьируется в диапазоне от 0.917 до 0.938, а максимальное отклонение от среднего значения составляет 1,93%.

Таким образом, наше первоначальное предположение не подтвердилось,  и на деле доход (категория дохода) не имеет значимого влияния на вероятность выплаты по кредиту в срок.

- Как разные цели кредита влияют на его возврат в срок?

Создадим сводную таблицу на основании результатов, полученных после категоризации данных:

In [46]:
purpose_pivot = solvency.pivot_table(values='debt', index='purpose', aggfunc=(sum, 'count')).reset_index()
purpose_pivot['index'] = 1 - purpose_pivot['sum'] / purpose_pivot['count']
purpose_pivot['delta%'] = (purpose_pivot['index'] - general_debt_index)*100
purpose_pivot

,purpose,count,sum,index,delta%
0,автомобиль,4306,403,0.906410,-1.243619
1,жилая недвижимость,9499,683,0.928098,0.925185
2,коммерческая недвижимость,1311,99,0.924485,0.563928
3,образование,4013,370,0.907800,-1.104620
4,свадьба,2324,186,0.919966,0.111973


### Вывод

Как мы видим, по категориям целей индекс совершения кредитного платежа в срок варьируется от 0.906 до 0.928, а максимальное среди категорий отклонение от среднего по всей выборке индекса составляет 1,24%.
На основании полученных данных можно заключить, что цель взятия кредита также не влияет в сколько-нибудь ощутимой степени на пунктуальность дебиторов при внесении кредитных платежей.

### Шаг 4. Общий вывод

Соберем для наглядности полученные выводы по вопросам Шага №3 в таблицу:


In [47]:
factor_list = [['children', 92.5, 90.2], 
               ['family_status', 92.0, 91.5], 
               ['total_income', 92.9, 91.7], 
               ['purpose', 92.8, 90.6]]
final_table = pd.DataFrame(factor_list, columns = ['factor', 'max_index', 'min_index'])
final_table

,factor,max_index,min_index
0,children,92.5,90.2
1,family_status,92.0,91.5
2,total_income,92.9,91.7
3,purpose,92.8,90.6


Как мы видим, по четырем параметрам оценки (наличие детей, семейное положение, ежемесячный доход и цели взятия кредита) фактически отсутствует дифференциация по вероятности своевременных выплат по кредиту. Минимальное значение вероятности составляет 90.2% (у респондента четверо детей), максимальное - 92.9% (у респондента ежемесячный доход в диапазоне от 300 тысяч до 1 млн.), но в целом можно говорить о том, что ни один из проанализированных параметров не влияет в значительной степени на выплату кредитных платежей в срок.

Также можно сделать вывод, что для всех категорий параметров и для выборки в целом 9 из 10 респондентов вносят платежи по кредиту в срок, и только 1 респондент из 10 имеет просрочки по кредитным платежам.